<a href="https://colab.research.google.com/github/Kalyanb447-github/kalyan_ai_flutter_python/blob/main/create_flower_ai_model_from_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import os

In [5]:
_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

zip_file = tf.keras.utils.get_file(origin=_URL,fname='flower_photos.tgz',extract=True,cache_subdir='/content')

base_dir=os.path.join(os.path.dirname(zip_file),'flower_photos')

228827136/228813984 [==============================] - 1s 0us/step


In [27]:
IMAGES_SIZE=224
BATCH_SIZE=64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
  directory=  base_dir,
    target_size=(IMAGES_SIZE, IMAGES_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

val_genetator = datagen.flow_from_directory(
  directory=  base_dir,
    target_size=(IMAGES_SIZE, IMAGES_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)



Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [29]:
print(train_generator.class_indices)

labels='\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt','w') as f: 
  f.write(labels)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [38]:
IMAGE_SHAPE=(IMAGES_SIZE, IMAGES_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SHAPE,
    include_top=False,
    weights='imagenet'
)

9420800/9406464 [==============================] - 0s 0us/step


In [40]:
base_model.trainable=True;

In [41]:
model= tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32,3,activation='relu'),            
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5,activation='softmax')              
])

In [44]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [45]:
epochs =10

history=model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_genetator
    )

Epoch 1/10
46/46 [==============================] - 589s 13s/step - loss: 0.7610 - accuracy: 0.7669 - val_loss: 3.4774 - val_accuracy: 0.5486
Epoch 2/10
46/46 [==============================] - 570s 12s/step - loss: 0.2910 - accuracy: 0.9071 - val_loss: 2.0974 - val_accuracy: 0.6744
Epoch 3/10
46/46 [==============================] - 566s 12s/step - loss: 0.1744 - accuracy: 0.9435 - val_loss: 5.0309 - val_accuracy: 0.5759
Epoch 4/10
46/46 [==============================] - 577s 13s/step - loss: 0.2082 - accuracy: 0.9388 - val_loss: 4.5593 - val_accuracy: 0.6471
Epoch 5/10
46/46 [==============================] - 580s 13s/step - loss: 0.1827 - accuracy: 0.9445 - val_loss: 5.0648 - val_accuracy: 0.5964
Epoch 6/10
46/46 [==============================] - 582s 13s/step - loss: 0.1310 - accuracy: 0.9575 - val_loss: 7.3243 - val_accuracy: 0.4897
Epoch 7/10
46/46 [==============================] - 581s 13s/step - loss: 0.1024 - accuracy: 0.9656 - val_loss: 10.3696 - val_accuracy: 0.4254
Epoch

In [47]:
saved_model_dir=''
tf.saved_model.save(model,saved_model_dir)

converter =tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

tflite_model=converter.convert()

with open('model.tflite','wb') as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: /tmp/tmpk6kshsr8/assets


INFO:tensorflow:Assets written to: /tmp/tmpk6kshsr8/assets


In [51]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>